In [24]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model  # Charger ton modèle personnalisé
from tensorflow.keras.preprocessing import image

# Charger le modèle personnalisé (remplace le chemin par celui où ton modèle est sauvegardé)
model = load_model('dermato_model.h5')  # Remplace ce chemin par ton propre modèle

# Définir les classes de ton modèle (Adapte les classes selon tes besoins)
classes = ['Healthy', 'Cowpox','Monkeypox','HFMD','Measles','Chickenpox']  # Remplace ou ajoute d'autres classes si nécessaire

# Ouvre la caméra (généralement la première caméra est indexée à 0)
cap = cv2.VideoCapture(0)

# Vérifie si la caméra a été ouverte correctement
if not cap.isOpened():
    print("Erreur: Impossible d'ouvrir la caméra.")
    exit()

while True:
    # Capture une image à partir de la caméra
    ret, frame = cap.read()

    # Vérifie si l'image a été capturée correctement
    if not ret:
        print("Erreur: Impossible de capturer l'image.")
        break

    # Convertir l'image BGR (OpenCV) en RGB
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Amélioration de l'image: égalisation d'histogramme (pour une meilleure visibilité sous différents éclairages)
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    img_eq = cv2.equalizeHist(img_gray)

    # Redimensionner l'image pour qu'elle corresponde à l'entrée de ton modèle (224x224)
    img_resized = cv2.resize(img_eq, (224, 224))  # Assurez-vous que les dimensions correspondent à ce que votre modèle attend
    
    # Convertir l'image en 3 canaux (RGB) si nécessaire
    img_resized_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)  # Convertir en RGB si l'image est en niveaux de gris
    
    # Ajouter une dimension batch et normaliser les pixels
    img_array = np.expand_dims(img_resized_rgb, axis=0)  # Ajouter une dimension pour le batch (1 image)
    img_preprocessed = img_array / 255.0  # Normalisation des pixels (si nécessaire)
    
    # Faire la prédiction sur l'image capturée
    preds = model.predict(img_preprocessed)

    # Obtenir les probabilités pour chaque classe
    class_probs = preds[0]

    # Afficher les probabilités pour chaque classe dans la console
    print("Probabilités pour chaque classe :")
    for i, prob in enumerate(class_probs):
        print(f"{classes[i]}: {prob:.2f}")
    
    # Trouver la classe avec la probabilité la plus élevée
    predicted_class = np.argmax(preds)  # L'indice de la classe avec la probabilité la plus élevée
    predicted_prob = np.max(preds)  # La probabilité associée à la classe prédite

    # Gestion du seuil de confiance
    if predicted_prob > 0.7:
        label = f"{classes[predicted_class]}: {predicted_prob:.2f}"
    else:
        label = "Inconnu ou incertain"

    # Afficher la classe et la probabilité sur l'image
    cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Affiche l'image capturée dans une fenêtre nommée 'Webcam'
    cv2.imshow('Webcam', frame)

    # Attendre l'appui sur la touche 'q' pour quitter
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Relâche la caméra et ferme les fenêtres OpenCV
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 579ms/step
Probabilités pour chaque classe :
Healthy: 0.00
Cowpox: 0.00
Monkeypox: 0.01
HFMD: 0.98
Measles: 0.00
Chickenpox: 0.01
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Probabilités pour chaque classe :
Healthy: 0.00
Cowpox: 0.01
Monkeypox: 0.06
HFMD: 0.91
Measles: 0.00
Chickenpox: 0.02
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Probabilités pour chaque classe :
Healthy: 0.00
Cowpox: 0.02
Monkeypox: 0.04
HFMD: 0.93
Measles: 0.00
Chickenpox: 0.02
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Probabilités pour chaque classe :
Healthy: 0.00
Cowpox: 0.01
Monkeypox: 0.03
HFMD: 0.94
Measles: 0.00
Chickenpox: 0.01
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Probabilités pour chaque classe :
Healthy: 0.00
Cowpox: 0.01
Monkeypox: 0.02
HFMD: 0.95
Measles: 0.00
Chickenpox: 0.01
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Probabilités pour chaque classe :
Healthy: 0.00
Cowpox: 0.01
Monkeypox: 0.03
HFMD: 0.95
Measles: 0.00
Chickenpox: 0.01
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Probabilités pour c

In [ ]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model

# Charger ton modèle
model = load_model('dermato_model.h5')

# Définir les classes
classes = ['Healthy', 'Cowpox', 'Monkeypox', 'HFMD', 'Measles', 'Chickenpox']

# Ouvrir la caméra
cap = cv2.VideoCapture(0)

# Vérifier si la caméra est ouverte
if not cap.isOpened():
    print("Erreur: Impossible d'ouvrir la caméra.")
    exit()

# Démarrer le timer
start_time = time.time()
duration = 30  # Durée en secondes avant de capturer

predicted_label = "Aucune prédiction faite"

while True:
    # Lire l'image de la caméra
    ret, frame = cap.read()
    if not ret:
        print("Erreur: Impossible de capturer l'image.")
        break

    height, width, _ = frame.shape

    # Définir une zone centrale de focus
    center_x, center_y = width // 2, height // 2
    roi_size = 150  # Taille du carré de focus
    top_left_x = max(center_x - roi_size, 0)
    top_left_y = max(center_y - roi_size, 0)
    bottom_right_x = min(center_x + roi_size, width)
    bottom_right_y = min(center_y + roi_size, height)

    # Dessiner le carré pour guider
    cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 255, 0), 2)

    # Afficher le timer restant sur l'écran
    elapsed_time = time.time() - start_time
    remaining_time = max(0, int(duration - elapsed_time))
    cv2.putText(frame, f"Detection dans : {remaining_time}s", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Montrer la vidéo
    cv2.imshow('Webcam - Focus sur la lésion', frame)

    # Vérifier si le temps est écoulé
    if elapsed_time >= duration:
        # Découper la région d'intérêt
        roi = frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x]

        # Traitement d'image
        roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        roi_gray = cv2.cvtColor(roi_rgb, cv2.COLOR_RGB2GRAY)
        roi_eq = cv2.equalizeHist(roi_gray)
        roi_eq_rgb = cv2.cvtColor(roi_eq, cv2.COLOR_GRAY2RGB)
        roi_resized = cv2.resize(roi_eq_rgb, (224, 224))
        roi_normalized = roi_resized / 255.0
        roi_input = np.expand_dims(roi_normalized, axis=0)

        # Prédiction finale
        preds = model.predict(roi_input)
        class_probs = preds[0]

        predicted_class = np.argmax(class_probs)
        predicted_prob = np.max(class_probs)

        if predicted_prob > 0.7:
            predicted_label = f"Prediction finale : {classes[predicted_class]} ({predicted_prob:.2f})"
        else:
            predicted_label = "Prediction finale : Inconnu ou incertain"

        break

    # Quitter si on appuie sur 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        predicted_label = "Opération annulée par l'utilisateur."
        break

# Fermer la caméra
cap.release()
cv2.destroyAllWindows()

# Afficher la prédiction finale dans la console
print("\n========== Résultat ==========")
print(predicted_label)
print("================================\n")
